<h1>
<center>
Midterm 2
</center>
</h1>
<div class=h1_cell>
<p>
There are 2 parts on the midterm, 50 points each. Each part has 3 questions. There is also an extra credit problem at the end.
<p>
We will use the Titanic table for this midterm. I have a wrangled version of it on google sheets that I set up for you.
<p>
</div>

In [0]:
import pandas as pd


titanic_table = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vR2XbLnUuYlxaFMdS8_bBX3iKUIDEii6Lg5Rxesf-Oh8a6z8-vAN6UDGejaOrBg5130h4O_dLkecKWQ/pub?output=csv')



In [0]:
pd.set_option('display.max_columns', None)

In [3]:
titanic_table.head(1)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,no_age,filled_age,emb_C,emb_Q,emb_S,emb_nan,age_bin,age_Child,age_Adult,age_Senior,sex_female,sex_male,ok_child,pclass_1,pclass_2,pclass_3,pclass_nan
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S,0,22.0,0,0,1,0,Child,1,0,0,0,1,0,0,0,1,0


In [4]:
!rm library_w19_week7.py

rm: cannot remove 'library_w19_week7.py': No such file or directory


In [5]:
from google.colab import files
files.upload()

Saving library_w19_week7.py to library_w19_week7.py


{'library_w19_week7.py': b'import pandas as pd\r\nimport numpy as np\r\nfrom functools import reduce\r\nfrom types import SimpleNamespace\r\nimport random\r\nimport matplotlib.pyplot as plt  \r\nimport operator\r\n\r\ndef predictor_case(row, pred, target):\r\n  case_dict = {(0,0): \'true_negative\', (1,1): \'true_positive\', (0,1): \'false_negative\', (1,0): \'false_positive\'}\r\n  actual = row[target]\r\n  prediction = row[pred]\r\n  case = case_dict[(prediction, actual)]\r\n  return case\r\n\r\ndef informedness(cases):\r\n  tp = 0\r\n  if \'true_positive\' in cases:\r\n    tp = cases[\'true_positive\']\r\n  tn = 0\r\n  if \'true_negative\' in cases:\r\n    tn = cases[\'true_negative\']\r\n  fp = 0\r\n  if \'false_positive\' in cases:\r\n    fp = cases[\'false_positive\']\r\n  fn = 0\r\n  if \'false_negative\' in cases:\r\n    fn = cases[\'false_negative\']\r\n  if (((tp+fn) == 0) or ((tn+fp) == 0)):\r\n    return -1\r\n  else:\r\n    recall = 1.0*tp/(tp+fn)\r\n    specificity = 1.0*

In [6]:
from library_w19_week7 import *

%who function

accuracy	 build_pred	 build_tree_iter	 compute_prediction	 compute_training	 euclidean_distance	 f1	 find_best_splitter	 forest_builder	 
forest_scores	 generate_table	 gig	 gini	 informedness	 k_fold	 k_fold_random	 knn	 knn_tester	 
oob_forest_scores	 oob_test	 oob_vote_taker	 path_id	 predictor_case	 probabilities	 produce_scores	 reorder_paths	 shuffle	 
tree_predictor	 verify_unique	 vote_taker	 


<h2>Columns used in all questions</h2>

We will use 14 columns from the table to make predictions on `Survived`.

In [0]:
usable_columns = [
 'emb_C',
 'emb_Q',
 'emb_S',
 'emb_nan',
 'age_Child',
 'age_Adult',
 'age_Senior',
 'no_age',
 'ok_child',
 'sex_female',
 'pclass_1',
 'pclass_2',
 'pclass_3',
 'pclass_nan'
]

target = 'Survived'

<h1>
Part 1. Explore random forest behavior
</h1>
<div class=h1_cell>
<p>
For prediction purposes, we have been treating a forest as a single predictor. It does call on its trees to get a prediction, but we don't see that. All we see is the final prediction. I'd like to dig deeper into individual tree behavior. I'll ask you to complete a set of programming problems to do this.
</div>

<h2>
Problem 1. Build a matrix of tree predictions (10 points)
</h2>
<div class=h1_cell>
<p>
The matrix will be a list of lists. Let's say the forest we are working with has 11 trees. Then the first list will be the predictions of the 11 trees for row 0 of the Titanic table, i.e., it will be a list of 11 binary values. The next list will be the 11 predictions for row 1 of the Titanic table. Given that we have 891 rows in the Titanic table, we will have 891 lists in our outer list.
<p>
Let's say I store my matrix in `all_trees`. Then `all_trees[i][j]` will represent the vote of the jth tree for row i.
  <p>
    Here is what you should see for the first 10 rows.
<p>
<pre>
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1],
 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0],
 [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1]]
 </pre>

</div>

<h2>
Here is forest to test on
</h2>
<div class=h1_cell>
<p>
  But first set random seeds so you get same results as mine.
</div>

In [0]:
import random

rng = np.random.RandomState(24)  #Will pass as arg to pandas sample method
random.seed(2000)

In [9]:
forest11 = forest_builder(titanic_table, usable_columns, target, hypers={'total-trees':11, 'random-state':rng})
len(forest11)

11

Now build the matrix and check your results against mine above.

In [69]:
all_trees = []
for index, row in titanic_table.iterrows():
  temp_list = []
  for tree in forest11:
    prediction = tree_predictor(row,tree)
    temp_list.append(prediction)
  all_trees.append(temp_list)
all_trees[:10]
len(all_trees[0])

11

<h1>
Problem 2: The most correct tree (10 points)
</h1>
<div class=h1_cell>
<p>
Produce an ordered list, from most correct to least correct, of the 11 trees taken over the 891 rows. A tree is correct if its prediction matches the `Survived` column in the target row. Here are my results.
  <pre>
[(2, 707),
 (1, 701),
 (10, 701),
 (4, 674),
 (6, 643),
 (7, 637),
 (3, 613),
 (0, 595),
 (5, 581),
 (8, 577),
 (9, 574)]
 </pre>
  The winner is tree2. It was correct 707 out of 891 times.
</div>

In [71]:
for i in range(len(all_trees)):
  for j in range(len(forest11)):
    if all_trees[i][j] == titanic_table.iloc[i]['Survived']:
      print(j, 1)

0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
10 1
0 1
1 1
2 1
4 1
5 1
7 1
8 1
10 1
1 1
4 1
6 1
0 1
1 1
2 1
4 1
6 1
7 1
9 1
10 1
0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
10 1
0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
10 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
10 1
0 1
1 1
3 1
4 1
6 1
7 1
9 1
10 1
1 1
4 1
6 1
0 1
1 1
2 1
3 1
4 1
5 1
6 1
8 1
10 1
1 1
4 1
5 1
6 1
8 1
0 1
1 1
2 1
4 1
6 1
7 1
9 1
10 1
0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
10 1
0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
10 1
0 1
2 1
3 1
5 1
7 1
8 1
9 1
10 1
0 1
1 1
2 1
3 1
4 1
6 1
10 1
0 1
1 1
3 1
4 1
5 1
7 1
10 1
0 1
0 1
2 1
3 1
5 1
7 1
8 1
9 1
10 1
1 1
5 1
8 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
10 1
0 1
1 1
4 1
6 1
0 1
9 1
0 1
2 1
3 1
7 1
9 1
10 1
1 1
4 1
6 1
0 1
1 1
2 1
3 1
4 1
6 1
7 1
9 1
10 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
10 1
1 1
4 1
0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
10 1
1 1
2 1
3 1
6 1
7 1
9 1
10 1
0 1
1 1
2 1
5 1
7 1
8 1
10 1
1 1
4 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
10 1
1 1
2 1
3 1
6 1
7 1
9 1
10 1
1 1
2 1
3 1
4 1
5 1


In [73]:
ol = []
for i in range(len(all_trees)):
  for j in range(len(forest11)):
    if all_trees[i][j] == titanic_table.iloc[i]['Survived']:
      ol.append((j, 1))
ol.sort(key = lambda x: x[1])
ol

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (0, 1),
 (1, 1),
 (2, 1),
 (4, 1),
 (5, 1),
 (7, 1),
 (8, 1),
 (10, 1),
 (1, 1),
 (4, 1),
 (6, 1),
 (0, 1),
 (1, 1),
 (2, 1),
 (4, 1),
 (6, 1),
 (7, 1),
 (9, 1),
 (10, 1),
 (0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (10, 1),
 (0, 1),
 (1, 1),
 (3, 1),
 (4, 1),
 (6, 1),
 (7, 1),
 (9, 1),
 (10, 1),
 (1, 1),
 (4, 1),
 (6, 1),
 (0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (8, 1),
 (10, 1),
 (1, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (8, 1),
 (0, 1),
 (1, 1),
 (2, 1),
 (4, 1),
 (6, 1),
 (7, 1),
 (9, 1),
 (10, 1),
 (0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),


<h1>
Problem 3: voting blocks (30 points)
</h1>
<div class=h1_cell>
<p>
Taking trees pairwise, produce an ordered list of which trees vote the same (right or wrong) most often.
  <p>
    For each specific row:
    <pre>
 1. if (tree1,tree2) vote the same then I record that.
 2. if (tree1,tree3) vote the same then I record that.
 etc
</pre>
I do that for the entire table. Here is first part of sorted list I got for forest11.
<pre>
[[((5, 8), 887),
 ((2, 10), 863),
 ((7, 10), 815),
 ((1, 4), 810),
 ((3, 10), 797),
 ((2, 7), 787),
 ((1, 6), 785),
 ((4, 6), 780),
 ((2, 3), 769),
 ((7, 9), 754)]
</pre>
  So tree5 and tree8 vote the same (right or wrong) 887 out of 891 times.
</div>

In [0]:
def oob_vote_taker2(row, forest):
    votes = {0:0, 1:0}
    for tree in forest:
        # check to see if it is in oob
        if row['index'] in tree['oob']:
            prediction = tree_predictor(row, tree)
            votes[prediction] += 1
    winner = 1 if votes[1]>votes[0] else 0  #ties go to 0
    return winner

[((5, 8), 887),
 ((2, 10), 863),
 ((7, 10), 815),
 ((1, 4), 810),
 ((3, 10), 797),
 ((2, 7), 787),
 ((1, 6), 785),
 ((4, 6), 780),
 ((2, 3), 769),
 ((7, 9), 754)]

It looks like for 891 rows, trees 5 and 8 vote the same in 887 of them. So they vote the same except for 4 rows.

<h1>
Part 2. Explore knn behavior
</h1>
<div class=h1_cell>
<p>
I'd like more information on how the rows are selected and how they vote.
  <p>
    Note for each of these problems I do not care what the actual prediction is. I am not computing accuracy but instead the internal working of the rows in voting.
    <p>
      <p>
Given the constrained time of the midterm, I am going to slice off 100 rows and just work with that.
</div>

In [0]:
small_titanic = titanic_table[:100]  #use this slice in your code to cut down on computing time

<h1>
Problem 4: A row's top-k counts (20 points)
</h1>
<div class=h1_cell>
<p>
For each row, count how many times it made the top-k. Store your answer in top_list in sorted fashion. Here are the first part of my results
  <pre>
[(0, 30),
 (12, 28),
 (4, 27),
 (13, 27),
 (37, 27),
 (8, 19),
 (29, 19),
 (45, 19),
 (51, 19),
 (59, 19)]
 </pre>
  So row 0 was among the top-k chosen 30 times out of 100 rows.
</div>

<h3>Hint</h3>

For this problem, and really most of the problems dealing with KNN, I started with the knn function and then modified it to do what is called for. For this problem, what is called for is a modified version of knn that does not return a prediction but instead the k rows selected to vote, i.e. the top k.
<p>
  Once I had the modified function, I called it in a loop that iterated over all the row indices of the Titanic table. The loop body updated a global data structure. At the end I got it in sorted list form and was finished.

In [0]:
def knn3(row_index,table,columns,k,target):#provide prediction for row given K-NN algorithm. Only use values in columns.
  table2 = table.copy()
  distances = []
  table2 = table.as_matrix(columns=columns)
  row1 = table2[row_index]
  target_votes = table[target].tolist()
  for item in range(len(table)):
    if item == row_index:
      continue
    distances.append((item,euclidean_distance(row1,table2[item])))
  distances.sort(key=operator.itemgetter(1))

  votes = []
  for (i,j) in distances[:k]:
    votes.append(target_votes[i])
  

  #  votes.append(table.iloc[i][target])
  return votes, distances

In [51]:
result = []
for i in range(len(small_titanic)):
  result.append((knn3(i, small_titanic, usable_columns, 10, target)))
result

[([0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [(12, 0.0),
   (37, 0.0),
   (51, 0.0),
   (59, 0.0),
   (67, 0.0),
   (69, 0.0),
   (75, 0.0),
   (80, 0.0),
   (86, 0.0),
   (89, 0.0),
   (91, 0.0),
   (93, 0.0),
   (2, 1.0),
   (7, 1.0),
   (14, 1.0),
   (38, 1.0),
   (49, 1.0),
   (50, 1.0),
   (63, 1.0),
   (68, 1.0),
   (71, 1.0),
   (4, 1.4142135623730951),
   (10, 1.4142135623730951),
   (13, 1.4142135623730951),
   (24, 1.4142135623730951),
   (27, 1.4142135623730951),
   (60, 1.4142135623730951),
   (72, 1.4142135623730951),
   (73, 1.4142135623730951),
   (74, 1.4142135623730951),
   (81, 1.4142135623730951),
   (90, 1.4142135623730951),
   (94, 1.4142135623730951),
   (8, 1.7320508075688772),
   (16, 1.7320508075688772),
   (18, 1.7320508075688772),
   (22, 1.7320508075688772),
   (25, 1.7320508075688772),
   (29, 1.7320508075688772),
   (39, 1.7320508075688772),
   (40, 1.7320508075688772),
   (44, 1.7320508075688772),
   (45, 1.7320508075688772),
   (56, 1.7320508075688772),
   (76, 1

<h1>
Problem 5: A row's correct count (20 points)
</h1>
<div class=h1_cell>
<p>
For each row, count how many times it was correct. Store your result in correct_list, in sorted fashion. Here are the first part of my results.
  <pre>
[(0, 25),
 (12, 25),
 (37, 25),
 (4, 19),
 (13, 19),
 (51, 19),
 (59, 19),
 (67, 18),
 (69, 14),
 (8, 13)]
 </pre>
  So when row 0 was in the top-k, it was correct 25 times.
</div>

[(0, 25),
 (12, 25),
 (37, 25),
 (4, 19),
 (13, 19),
 (51, 19),
 (59, 19),
 (67, 18),
 (69, 14),
 (8, 13)]

<h1>
Problem 6: Put them together to get each row's "batting average" (10 points)
</h1>
<div class=h1_cell>
<p>
Use `top_list` and `correct_list` to compute what percentage each row was correct, i.e., divide times correct by total times in top-k. Then sort the list by highest average. When I did that I got results below taken from the slice ` [50:80]`.  I am showing my results more toward the middle to  give you a better answer to shoot for. REMEMBER: this is for items 50:80.
  <pre>
[(82, 0.6),
 (85, 0.6),
 (62, 0.5833333333333334),
 (7, 0.5714285714285714),
 (31, 0.5714285714285714),
 (48, 0.5714285714285714),
 (92, 0.5714285714285714),
 (20, 0.5625),
 (53, 0.5555555555555556),
 (66, 0.5555555555555556),
 (1, 0.5454545454545454),
 (68, 0.5454545454545454),
 (30, 0.5),
 (34, 0.5),
 (35, 0.5),
 (64, 0.5),
 (83, 0.5),
 (88, 0.5),
 (96, 0.5),
 (98, 0.5),
 (70, 0.4444444444444444),
 (99, 0.4444444444444444),
 (11, 0.42857142857142855),
 (5, 0.4),
 (21, 0.4),
 (27, 0.4),
 (40, 0.4),
 (46, 0.4),
 (71, 0.4),
 (23, 0.375)]
 </pre>
  So row 82 is batting 60%. When I divide the number of times it voted correctly (in `correct_list`) by the number of times it made the top-k (in `top_list`), it was correct .6 (60%) of the time. It has a 600 batting average!
</div>

<h3>Hint</h3>

For this problem I did not define a new function. I just figured out a way to get info out of top_list and correct_list to get what I wanted.

In [0]:
#remember that you should show rows 50:80


[(82, 0.6),
 (85, 0.6),
 (62, 0.5833333333333334),
 (7, 0.5714285714285714),
 (31, 0.5714285714285714),
 (48, 0.5714285714285714),
 (92, 0.5714285714285714),
 (20, 0.5625),
 (53, 0.5555555555555556),
 (66, 0.5555555555555556),
 (1, 0.5454545454545454),
 (68, 0.5454545454545454),
 (30, 0.5),
 (34, 0.5),
 (35, 0.5),
 (64, 0.5),
 (83, 0.5),
 (88, 0.5),
 (96, 0.5),
 (98, 0.5),
 (70, 0.4444444444444444),
 (99, 0.4444444444444444),
 (11, 0.42857142857142855),
 (5, 0.4),
 (21, 0.4),
 (27, 0.4),
 (40, 0.4),
 (46, 0.4),
 (71, 0.4),
 (23, 0.375)]

<h2>
Extra Credit: razor-thin majority
</h2>
<div class=h1_cell>
<p>
Find what target rows were decided by either 1 vote or were a tie. 
</div>

<h3>Hint</h3>

I went back to modifying knn for this problem. I replaced the knn function with a razor_thin function. It did much of the same work as knn, but instead of returning a prediction, it returned True or False. True if there was razor thin majority (or tie vote) and False otherwise. I then kept track of what rows I got a True for and sorted the list and was done.
<p>
  Caveat: I found no razor-thin votes so had an empty list. I even tried with entire titanic table and still got no razor thins. I wonder if I can put this off to "group think". Never seems to be a close call.

None
